# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
import os
pf.__version__

'0.0.58'

In [2]:
MasterFolder=r"C:\Users\charl\Documents\Instruments and calibrations"
DayFolder=MasterFolder + "/UCB_5-10-23"

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')
if not os.path.exists(MasterFolder + '\OUTPUT'+'\Pickles'):
    os.mkdir(MasterFolder + '\OUTPUT'+'\Pickles')

if not os.path.exists(DayFolder + '\GROUPS'):
    os.mkdir(DayFolder +'\GROUPS')

if not os.path.exists(DayFolder + '\FITOUT'):
    os.mkdir(DayFolder +'\FITOUT')


meta_path=DayFolder +  '\METADATA'
spectra_path=DayFolder + '\SPECTRA'

output_path=MasterFolder +"\OUTPUT"
fitout_path=DayFolder+'\FITOUT'
group_path=DayFolder+'\GROUPS'

## Load in specra, and combine

In [3]:
import os.path
from os import path

if os.path.exists(fitout_path+'/'+'Discarded_df.xlsx'):
    discard=pd.read_excel(fitout_path+'/'+'Discarded_df.xlsx')
else:
    discard=None
if os.path.exists(fitout_path+'/'+'Weak_Diads.xlsx'):
    grp1=pd.read_excel(fitout_path+'/'+'Weak_Diads.xlsx')
else:
    grp1=None
if os.path.exists(fitout_path+'/'+'Medium_Diads.xlsx'):
    grp2=pd.read_excel(fitout_path+'/'+'Medium_Diads.xlsx')
else:
    grp2=None
if os.path.exists(fitout_path+'/'+'Strong_Diads.xlsx'):
    grp3=pd.read_excel(fitout_path+'/'+'Strong_Diads.xlsx')
else:
    grp3=None

df2=pd.concat([grp1, grp2, grp3], axis=0).reset_index(drop=True).drop(columns=['Unnamed: 0'])

if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,Diad1_fwhm,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,02 C1,103.391008,1285.469804,4163.625107,1285.469854,7026.469367,0.630367,5.047749,0.635740,1.260734,...,0.747579,1370.688578,153.313244,0.264425,1381.895112,412.592568,13.221228,1278.518822,11.684789,15.742187
1,03 C2_CRR_DiadFit,103.393121,1285.469275,4171.082825,1285.469325,7017.968331,0.622779,4.594285,0.658053,1.245557,...,0.743428,1370.692906,149.765749,0.263972,1381.895112,428.922016,13.198592,1278.518822,5.494752,6.221358
2,05 C3,103.464399,1285.310237,4204.288726,1285.310287,7766.945874,0.687822,6.231493,0.645708,1.375644,...,0.796233,1370.632124,149.914621,0.283854,1381.757622,638.137352,14.192697,1278.326335,165.505721,17.155309
3,06 C4,103.466682,1285.303008,4227.475202,1285.303058,7799.535768,0.685365,6.613714,0.650898,1.370731,...,0.808257,1370.649363,150.445596,0.284631,1381.746622,683.457919,14.231573,1278.315336,136.699401,17.098658
4,07 C5,103.470923,1285.291018,4195.853316,1285.291068,7821.906713,0.693813,5.846498,0.646158,1.387625,...,0.826593,1370.651918,153.313247,0.283703,1381.735623,665.974855,14.185211,1278.287838,125.790720,17.311616


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window','IMG','nodiad'],
 sort=True)
diad_meta

['02 C1.txt',
 '03 C2.txt',
 '05 C3.txt',
 '06 C4.txt',
 '07 C5.txt',
 '10 C6.txt',
 '11 C7.txt',
 '12 C8.txt',
 '14 C9.txt',
 '15 C10.txt',
 '17 C11.txt',
 '19 C12.txt',
 '21 C13.txt',
 '23 C14.txt',
 '25 C15.txt',
 '27 C16.txt',
 '29 C17.txt',
 '31 C18.txt',
 '33 C19.txt',
 '35 C20.txt',
 '37 C21.txt',
 '39 C22.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 22/22 [00:00<00:00, 402.92it/s]

Done


'02 C1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'02 C1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['02 C1', '03 C2', '05 C3', '06 C4', '07 C5', '10 C6', '11 C7',
       '12 C8', '14 C9', '15 C10', '17 C11', '19 C12', '21 C13', '23 C14',
       '25 C15', '27 C16', '29 C17', '31 C18', '33 C19', '35 C20',
       '37 C21', '39 C22'], dtype='<U6')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
df_combo

,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,Diad1_fwhm,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,02 C1,103.391008,1285.469804,4163.625107,1285.469854,7026.469367,0.630367,5.047749,0.635740,1.260734,...,May,10.0,35.183,30.0,5.0,5.0,"['0h', '2m', '32s']",10:28:16 AM\n,37696.0,1325.004
1,03 C2_CRR_DiadFit,103.393121,1285.469275,4171.082825,1285.469325,7017.968331,0.622779,4.594285,0.658053,1.245557,...,May,10.0,35.136,30.0,5.0,5.0,"['0h', '2m', '31s']",10:31:04 AM\n,37864.0,1325.004
2,05 C3,103.464399,1285.310237,4204.288726,1285.310287,7766.945874,0.687822,6.231493,0.645708,1.375644,...,May,10.0,35.029,30.0,4.0,5.0,"['0h', '2m', '1s']",10:48:19 AM\n,38899.0,1325.004
3,06 C4,103.466682,1285.303008,4227.475202,1285.303058,7799.535768,0.685365,6.613714,0.650898,1.370731,...,May,10.0,35.061,30.0,4.0,5.0,"['0h', '2m', '1s']",10:51:00 AM\n,39060.0,1325.004
4,07 C5,103.470923,1285.291018,4195.853316,1285.291068,7821.906713,0.693813,5.846498,0.646158,1.387625,...,May,10.0,35.070,30.0,4.0,5.0,"['0h', '2m', '1s']",10:53:48 AM\n,39228.0,1325.004
5,10 C6,103.577333,1285.017781,4716.166451,1285.017831,9971.008125,0.784191,8.173181,0.657065,1.568383,...,May,10.0,35.277,30.0,4.0,5.0,"['0h', '2m', '1s']",11:23:47 AM\n,41027.0,1325.004
6,11 C7,103.578470,1285.010446,4792.232404,1285.010496,10156.515642,0.784476,8.049490,0.662687,1.568951,...,May,10.0,35.326,30.0,4.0,5.0,"['0h', '2m', '1s']",11:26:47 AM\n,41207.0,1325.004
7,12 C8,103.581851,1285.000793,4807.906997,1285.000843,10217.026896,0.789849,8.092965,0.653103,1.579698,...,May,10.0,35.345,30.0,4.0,5.0,"['0h', '2m', '1s']",11:29:48 AM\n,41388.0,1325.004
8,14 C9_CRR_DiadFit,103.588906,1284.978416,4589.174482,1284.978466,9908.445915,0.795605,7.792152,0.673438,1.591209,...,May,10.0,35.327,30.0,4.0,5.0,"['0h', '2m', '1s']",11:40:03 AM\n,42003.0,1325.004
9,15 C10_CRR_DiadFit,103.608082,1284.931800,4974.552644,1284.931850,10989.897159,0.810586,8.831108,0.684358,1.621171,...,May,10.0,35.333,30.0,4.0,5.0,"['0h', '2m', '1s']",11:50:12 AM\n,42612.0,1325.004


In [12]:
from pickle import load
import pickle
import os
nm=os.path.basename(DayFolder)

pickle_path=MasterFolder + '\OUTPUT'+'\Pickles'
file_pkl=pickle_path + '/' +'Ne_lineReg_'+nm+'_PVM.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [13]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [15]:
nm=os.path.basename(DayFolder)
df_combo.to_excel(output_path+'/'+nm+'_Calibration_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [16]:
if os.path.exists(fitout_path+'/'+'Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel(fitout_path+'/'+'Carb_Peak_fits.xlsx')
else:
    Carb=pd.DataFrame([])
if os.path.exists(fitout_path+'/'+'SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel(fitout_path+'/'+'SO2_Peak_fits.xlsx')
else:
    SO2=pd.DataFrame([])

if (Carb.empty==False) & (SO2.empty==False):
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
    
if (Carb.empty==True) & (SO2.empty==False):
    Sec_Phases=SO2
if (Carb.empty==False) & (SO2.empty==True):
    Sec_Phases=Carb
if (Carb.empty==True) & (SO2.empty==True):
    Sec_Phases=pd.DataFrame([]) 
    print('You have no secondary phases!')

if Sec_Phases.empty==False:
    display(Sec_Phases.head())    

You have no secondary phases!


In [17]:
# Remove these to get the pure file name
if Sec_Phases.empty==False:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    display(file_sec_phase)

In [18]:
if Sec_Phases.empty==False:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']
    df_combo_sec_phase=df_combo.merge(Sec_Phases, on='name_for_matching', how='outer')
else:
    df_combo_sec_phase=df_combo

### Add some more useful columns

In [19]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [20]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_Calibration_fitting_w_sec_pg.xlsx')
os.system('echo -en "\a"')
print('done')

done
